Welcome to Python world and Bertopic modeling for Master Data Incidents Analysis! Ensure you have set up your environment as instructions and if so, let's have fun!

# 00. Install Packages


In [193]:
#!pip install openpyxl
# pandas need openpyxl (to read excel file, if your file is in excel)
#!pip install pandas
#!pip install sentence-transformers
#!pip install bertopic
#may take around 30 seconds. will output requirement already satisfied or installed successfully when finalized


In [194]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: openpyxl in c:\users\uy.lr\appdata\local\programs\python\python312\lib\site-packages (3.1.3)




[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [195]:
pip install pandas


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [196]:
pip install sentence-transformers


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [197]:
pip install hdbscan

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [198]:
#pip cache purge

In [199]:
pip install bertopic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [200]:
pip install transformers[torch]


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import os
import numpy as np
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from sklearn.cluster import KMeans
#from sklearn import svc
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from torch import bfloat16


c:\Users\uy.lr\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# 0. Starting with no label

At the very first beginning we use a pure BERTopic model to have the first clustering without any guidence by labelling it with PRM Analysis. With the result of this version clustering, we generated PRM Analysis with review comments for better capability of fine tuning the model. I leave the codes here just in case you might wonder how we start initially.  
Can skip and jump to the second section

## 01. Data Preparation.
Go to ServiceNow to export your data in csv preferred, conditional filters of previously used data has been displayed in user guide.

In [2]:
file_path = 'incident_230630_240111.csv' #replace the file with your data file in csv - assumes same place as the python notebook
# please leave no blank in the title of the file, it will raise bug.
# the most convinent way is leave this ipynb with the files in the same location, so that you don't need relocate the file path to other places.
#always prefer csv rather than excel (excel needs closed when read the file)
data = pd.read_csv(file_path, encoding='latin1').apply(lambda x: x.astype(str)) 

# housekeeping: renames data columns into desired column names
data = data.rename(columns={'problem_id.u_component': 'problem_component'})
data = data.rename(columns={'inc_short_description': 'short_description'}) # 5/15: added from Alvin's extract

In [3]:
data #have a look of your data frame
#data.head() #this line gives you first 5 rows of the data frame, remove the first # in this line if you want to run it
data.columns

Index(['number', 'u_component', 'problem_id', 'short_description', 'priority',
       'active', 'opened_at', 'location', 'contact_type', 'state', 'closed_at',
       'u_causing_it_layer', 'u_symptom_type', 'u_preliminary_cause',
       'u_resolution_code', 'u_moss_closure_code', 'category', 'cmdb_ci',
       'u_impacted_business_process', 'u_pg_inc_cause_by_chg'],
      dtype='object')

In [4]:
description = data['short_description'].tolist() #input to model should be a list

In [205]:
# len(embeddings) # to check how long is the embedded vector (should be the same as row amount)

In [5]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(2,3)) # use bigram and trigram, which is combination of 2 or 3 words as the key words
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
cluster_model = KMeans(n_clusters=20) #use kmeans rather than hdbscan
     

In [6]:
from sentence_transformers import SentenceTransformer
 
model = SentenceTransformer("BAAI/bge-small-en-v1.5")

In [7]:
# DEFINING THE MODEL

topic_model = BERTopic(min_topic_size=2, language="english", calculate_probabilities=True,
                       embedding_model="BAAI/bge-small-en-v1.5",
                       umap_model=umap_model,
                       hdbscan_model=cluster_model,
                       vectorizer_model=vectorizer_model,
                        top_n_words=5,
                        verbose=True)

In [8]:
import requests
requests.get('https://www.huggingface.co')

<Response [200]>

In [9]:
# Train model
topics = topic_model.fit_transform(description)


2024-07-23 09:11:31,799 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 13/13 [00:04<00:00,  2.74it/s]
2024-07-23 09:11:39,922 - BERTopic - Embedding - Completed ✓
2024-07-23 09:11:39,922 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-23 09:11:45,527 - BERTopic - Dimensionality - Completed ✓
2024-07-23 09:11:45,527 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-23 09:11:46,237 - BERTopic - Cluster - Completed ✓
2024-07-23 09:11:46,237 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-23 09:11:46,290 - BERTopic - Representation - Completed ✓


In [10]:
topics_info = topic_model.get_topic_info()
topics_info #have a look of the topics and its key words. forget topic 0 since it's a 'other' collection for incidents cannot be clustered into other topics

,Topic,Count,Name,Representation,Representative_Docs
0,0,34,0_cancelled shipment_san martin_san martin obi...,"[cancelled shipment, san martin, san martin ob...",[[WMS] [PrIME-LA] [MDCLX7] [San Martin Obispo]...
1,1,30,1_wms prime_sap wms prime_sap wms_work order,"[wms prime, sap wms prime, sap wms, work order...",[[WMS] [PrIME-AMA-W] [HYDLX5/6] [Hyderabad] ...
2,2,30,2_master receipt_martin obispo_san martin_san ...,"[master receipt, martin obispo, san martin, sa...",[[WMS] [PrIME-NA] [DALLG5] [Dallas DLMC] DLMC ...
3,3,27,3_carrier id_obispo carrier_martin obispo carr...,"[carrier id, obispo carrier, martin obispo car...",[[WMS] [PrIME-LA] [MDCLX7] [San Martin Obispo]...
4,4,24,4_mobile id_unassigned quantity cancelled_unas...,"[mobile id, unassigned quantity cancelled, una...",[[WMS] [PrIME-GC] [HPLX5/6] [Huangpu] Shipment...
5,5,23,5_cycle count_wms prime ama_prime ama_wms prime,"[cycle count, wms prime ama, prime ama, wms pr...",[[WMS] [PrIME-AMA-E] [BNCLX6] [Ben Cat] Site u...
6,6,22,6_lvlx5 louveira_la lvlx5 louveira_la lvlx5_pr...,"[lvlx5 louveira, la lvlx5 louveira, la lvlx5, ...",[[WMS] [PrIME-GC] [LULX6] [Luogang] Shipment I...
7,7,22,7_line ship qty_ship qty_line ship_error line ...,"[line ship qty, ship qty, line ship, error lin...",[[WMS] [PrIME-GC] [HPLX5/6] [Huangpu] Shipment...
8,8,21,8_unallocated quantity wms_unallocated quantit...,"[unallocated quantity wms, unallocated quantit...",[[WMS] [PrIME-LA] [MDCLX8] [San Martin Obispo]...
9,9,21,9_valley wcmc_na mvlg5_prime na mvlg5_na mvlg5...,"[valley wcmc, na mvlg5, prime na mvlg5, na mvl...",[[WMS] [PrIME-NA] [MVLG5] [Moreno Valley WCMC]...


# 1. Fine tune with labelled data

In [11]:
file_path = 'WMS PRM Analysis.csv'
# please leave no blank in the title of the file, it will raise bug.
# the most convinent way is leave this ipynb with the files in the same location, so that you don't need relocate the file path to other places.
data = pd.read_csv(file_path,encoding='cp1252').reset_index() #always prefer csv rather than excel (excel needs closed when read the file)
training_data = data.dropna(subset=['H1']) #H1 column is the one with higher hierachy of clustering, including some detailed labels
#training_data.head() #- first five lines only

In [12]:

training_data.to_excel(
'training_data1.xlsx'
)

In [13]:
# CATEGORY NAMES
training_data['H1'].value_counts().sort_index()

H1
Directed work - Incorrect location                    1
Invalid Ticket - Should be FR                        13
Inventory Management - Cannot adjust inventory        1
Inventory Management - Committed Quantity Issue       1
Inventory Management - Delete load ID's               1
                                                     ..
Shipment - Update shipment status                     2
Shipment - Work queue issue                           2
Work queue - Directed work issue                      1
Work queue - Not sorted according to vehicle type     1
Work queue - PLL Item pick will not release\n         1
Name: count, Length: 109, dtype: int64

In [14]:
training_data['groups'] = training_data.groupby('H1').ngroup() #group index for higher layer

In [15]:
training_data['groups'].value_counts().sort_index()

groups
0       1
1      13
2       1
3       1
4       1
       ..
104     2
105     2
106     1
107     1
108     1
Name: count, Length: 109, dtype: int64

In [16]:
#overview of labels
category_names = training_data['H1'].unique()
category_names

array(['Shipment - Picked Quantity is the same with cancelled quantity Issue',
       'Related to PRB0108700',
       'Shipment - Cannot use location due to a source shipment that is already dispatched',
       'Receiving - duplicate GR IDOC posting',
       'Production - Closed work order showing in prime as open status',
       'Shipment - Picked Quantity Issue',
       'Invalid Ticket - Should be FR', 'SAP - SAP Incident',
       'Report - Order Content Check without order line details',
       'Shipment - Cannot close shipment', 'Shipment - Cannot allocate',
       'Directed work - Incorrect location',
       'Shipment - Released pick cancellation not showing in work queue',
       'Masterdata - Item Footprint (Outlier)', 'Related to PRB0104570',
       'Shipment - Shipment ID is loaded', 'Shipment - Error 11308',
       'Related to PRB0104294', 'Shipment - Overpicked ordered quantity',
       'Shipment - Cannot use location due to old replenishments/picks',
       'Receiving - Can

In [17]:
groups = training_data['groups'].to_numpy()
#groups



In [18]:
# #format variables to proper ones
docs = training_data['short_description'].to_numpy()
#docs

In [19]:
topic_model = BERTopic(verbose=True).fit(docs, y=groups)

2024-07-23 09:12:19,950 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 8/8 [00:00<00:00, 12.10it/s]
2024-07-23 09:12:23,954 - BERTopic - Embedding - Completed ✓
2024-07-23 09:12:23,954 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-23 09:12:25,403 - BERTopic - Dimensionality - Completed ✓
2024-07-23 09:12:25,404 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-23 09:12:25,420 - BERTopic - Cluster - Completed ✓
2024-07-23 09:12:25,421 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-23 09:12:25,421 - BERTopic - Representation - Completed ✓


# 3. Testing (to new incidents we have), by using the model finetuned

In [61]:
file_path2 = 'incident_sla-2324-onwards_715.csv' #replace the file by the incidents you want

#incident_sla-051524-1850
#incident_sla-2324-onwards_715
#incident_230630_240111

testing = pd.read_csv(file_path2, encoding='latin1').apply(lambda x: x.astype(str)) #,index_col='number')
testing = testing.rename(columns={'problem_id.u_component': 'problem_component', 'inc_short_description': 'short_description'})

In [264]:
#included in code above
#testing = testing.rename(columns={'inc_short_description': 'short_description'}) 

In [62]:
docs_testing = testing['short_description']

In [63]:
print(topic_model)

BERTopic(calculate_probabilities=False, ctfidf_model=ClassTfidfTransformer(...), embedding_model=<bertopic.backend._sentencetransformers.SentenceTransformerBackend object at 0x000001CBDF9F94C0>, hdbscan_model=HDBSCAN(...), language=english, low_memory=False, min_topic_size=10, n_gram_range=(1, 1), nr_topics=None, representation_model=None, seed_topic_list=None, top_n_words=10, umap_model=UMAP(...), vectorizer_model=CountVectorizer(...), verbose=True, zeroshot_min_similarity=0.7, zeroshot_topic_list=None)


In [64]:
topics, probs = topic_model.fit_transform(docs_testing)

2024-07-23 12:42:37,392 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 18/18 [00:04<00:00,  4.40it/s]
2024-07-23 12:42:41,504 - BERTopic - Embedding - Completed ✓
2024-07-23 12:42:41,504 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-23 12:42:44,251 - BERTopic - Dimensionality - Completed ✓
2024-07-23 12:42:44,263 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-23 12:42:44,295 - BERTopic - Cluster - Completed ✓
2024-07-23 12:42:44,295 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-23 12:42:44,342 - BERTopic - Representation - Completed ✓


In [65]:
result = topic_model.get_topic_info()
result

,Topic,Count,Name,Representation,Representative_Docs
0,-1,112,-1_primeeu_wms_nemc_shippensburg,"[primeeu, wms, nemc, shippensburg, shlg5, prim...",[[WMS] [PrIME-NA] [SHLG5] [Shippensburg NEMC] ...
1,0,88,0_rtcisna_rtciseu_rtcis_wms,"[rtcisna, rtciseu, rtcis, wms, rtcisla, rtcisa...",[[WMS] [RTCIS-NA] [GBLX4] [Green Bay] User aa3...
2,1,72,1_primena_greater_chicago_moreno,"[primena, greater, chicago, moreno, valley, wc...",[[WMS] [PrIME-NA] [MVLG5] [Moreno Valley WCMC]...
3,2,48,2_obispo_martin_san_mdclx7,"[obispo, martin, san, mdclx7, primela, carrier...",[[WMS] [PrIME-LA] [MDCLX7] [San Martin Obispo]...
4,3,22,3_louveira_lvlx5_primela_lvlx56,"[louveira, lvlx5, primela, lvlx56, the, shipme...",[[WMS] [PrIME-LA] [LVLX5] [Louveira] cannot cu...
5,4,22,4_trailer_move_chicago_greater,"[trailer, move, chicago, greater, to, gcfclg5,...",[[WMS] [PrIME-NA] [GCFCLG5] [Greater Chicago] ...
6,5,19,5_rhodonite_rholg5_andover_primena,"[rhodonite, rholg5, andover, primena, rholg567...",[[WMS] [PrIME-NA] [RHOLG5] [Andover Rhodonite]...
7,6,18,6_delete_unable_fulfillment_southeast,"[delete, unable, fulfillment, southeast, cente...",[[WMS] [PrIME-NA] [MVLG5] [Moreno Valley WCMC]...
8,7,18,7_pallet_pallets_gyongyos_received,"[pallet, pallets, gyongyos, received, karachi,...",[[WMS] [PrIME-EU] [GYOLG5] [Gyongyos] Load are...
9,8,17,8_master_receipt_obispo_martin,"[master, receipt, obispo, martin, san, primela...",[[WMS] [PrIME-LA] [MDCLX7] [San Martin Obispo]...


In [57]:
print(topic_model)

BERTopic(calculate_probabilities=False, ctfidf_model=ClassTfidfTransformer(...), embedding_model=<bertopic.backend._sentencetransformers.SentenceTransformerBackend object at 0x000001CBDF9F94C0>, hdbscan_model=HDBSCAN(...), language=english, low_memory=False, min_topic_size=10, n_gram_range=(1, 1), nr_topics=None, representation_model=None, seed_topic_list=None, top_n_words=10, umap_model=UMAP(...), vectorizer_model=CountVectorizer(...), verbose=True, zeroshot_min_similarity=0.7, zeroshot_topic_list=None)


In [ ]:
# result.to_excel('lexine3.xlsx')

In [262]:
testing['GROUPS'] = topics

In [ ]:
#testing.to_excel('lexine1.xlsx') #change the name to your desired one, download it to local 

To save trained model (pickle version)

In [88]:
import pickle

# Save model in pickle file

filename = open("final_trained_model.pkl", 'wb')
pickle.dump(topic_model, filename)
filename.close()


In [89]:

filename = "final_trained_model.pkl"
input_file_path = 'incident_sla-2324-onwards_715.csv'  # replace the file by the incidents you want

# Load the model
loaded_model = pickle.load(open(filename, 'rb')) 
#pickle.dump(loaded_model, open(filename, 'wb'))

# Read and process the input data
input_data = pd.read_csv(input_file_path, encoding='latin1').apply(lambda x: x.astype(str)).rename(columns={'problem_id.u_component': 'problem_component', 'inc_short_description': 'short_description'})
docs_processing = input_data['short_description']

# Fit the model and obtain topics and probabilities
topics, probs = loaded_model.fit_transform(docs_processing)


# Get topic information
result = loaded_model.get_topic_info()
result


2024-07-23 13:24:27,517 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 18/18 [00:03<00:00,  4.62it/s]
2024-07-23 13:24:31,431 - BERTopic - Embedding - Completed ✓
2024-07-23 13:24:31,431 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-23 13:24:34,235 - BERTopic - Dimensionality - Completed ✓
2024-07-23 13:24:34,235 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-23 13:24:34,254 - BERTopic - Cluster - Completed ✓
2024-07-23 13:24:34,256 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-23 13:24:34,282 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,135,-1_wms_primena_not_shippensburg,"[wms, primena, not, shippensburg, nemc, shlg5,...",[[WMS] [PrIME-NA] [SHLG5] [Shippensburg NEMC] ...
1,0,91,0_rtcisna_rtciseu_rtcis_wms,"[rtcisna, rtciseu, rtcis, wms, rtcisla, rtcisa...",[[WMS] [RTCIS-NA] [GBLX4] [Green Bay] User aa3...
2,1,37,1_obispo_martin_san_primela,"[obispo, martin, san, primela, mdclx7, carrier...",[[WMS] [PrIME-LA] [MDCLX7] [San Martin Obispo]...
3,2,34,2_moreno_valley_mvlg5_wcmc,"[moreno, valley, mvlg5, wcmc, primena, close, ...",[[WMS] [PrIME-NA] [MVLG5] [Moreno Valley WCMC]...
4,3,30,3_delete_jijona_jijlg5_primeeu,"[delete, jijona, jijlg5, primeeu, unable, orde...",[[WMS] [PrIME-EU] [JIJLG5] [Jijona] PrIME retu...
5,4,21,4_trailer_move_greater_chicago,"[trailer, move, greater, chicago, to, gcfclg5,...",[[WMS] [PrIME-NA] [GCFCLG5] [Greater Chicago] ...
6,5,21,5_louveira_lvlx5_primela_lvlx56,"[louveira, lvlx5, primela, lvlx56, shipment, l...",[[WMS] [PrIME-LA] [LVLX5] [Louveira] cannot cu...
7,6,20,6_greater_chicago_gcfclg5_primena,"[greater, chicago, gcfclg5, primena, shipment,...",[[WMS] [PrIME-NA] [GCFCLG5] [Greater Chicago] ...
8,7,20,7_pallet_pallets_primeeu_gyongyos,"[pallet, pallets, primeeu, gyongyos, received,...",[[WMS] [PrIME-EU] [GYOLG5] [Gyongyos] Load are...
9,8,18,8_master_receipt_obispo_martin,"[master, receipt, obispo, martin, san, primela...",[[WMS] [PrIME-LA] [MDCLX7] [San Martin Obispo]...


To test trained model 

In [87]:
import pickle

# Save model in pickle file

filename = open("new_trained_model6.pkl", 'wb')
pickle.dump(topic_model, filename)
filename.close()


input_file_path = 'incident_sla-051524-1850.csv'

#incident_sla-051524-1850
#incident_sla-2324-onwards_715
#incident_230630_240111

model_file = open('new_trained_model6.pkl', 'rb')

loaded_model = pickle.load(model_file)
model_file.close()



input_data = pd.read_csv(input_file_path, encoding='latin1').apply(lambda x: x.astype(str)).rename(columns={'problem_id.u_component': 'problem_component', 'inc_short_description': 'short_description'})


    # Fit the model and obtain topics and probabilities
docs_processing = input_data['short_description']
topics, probs = loaded_model.fit_transform(docs_processing)


    # Get topic information
result = loaded_model.get_topic_info()

print(result)


2024-07-23 13:07:42,714 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 139/139 [00:33<00:00,  4.13it/s]
2024-07-23 13:08:16,452 - BERTopic - Embedding - Completed ✓
2024-07-23 13:08:16,452 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-23 13:08:25,333 - BERTopic - Dimensionality - Completed ✓
2024-07-23 13:08:25,349 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-23 13:08:25,481 - BERTopic - Cluster - Completed ✓
2024-07-23 13:08:25,481 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-23 13:08:25,626 - BERTopic - Representation - Completed ✓


hi
     Topic  Count                                    Name  \
0       -1    976                 -1_rtcisna_in_wms_error   
1        0    121        0_trailer_close_dispatch_gcfclg5   
2        1    107                  1_sap_idoc_prime_idocs   
3        2    105               2_sap_idocs_flowing_rtcis   
4        3     94          3_pallets_pallet_amplg5_kheops   
..     ...    ...                                     ...   
104    103     11         103_taicang_batch_tclx3_rtcisgc   
105    104     11      104_gyolg5_gyongyos_agains_working   
106    105     11              105_blois_fert_bllx5_bllx6   
107    106     10  106_slowness_cdlx5_performance_cabuyao   
108    107     10      107_password_change_swarm_changing   

                                        Representation  \
0    [rtcisna, in, wms, error, not, rtciseu, to, rt...   
1    [trailer, close, dispatch, gcfclg5, trailers, ...   
2    [sap, idoc, prime, idocs, flowing, from, not, ...   
3    [sap, idocs, flowing, rtcis